In [1]:
import numpy as np
from random import randint
from scipy.optimize import fsolve
from utils.demand_functions import LogitDemand
demandfn = LogitDemand({
    "a0":0,
    "cost": 1,
    "mu":0.25,
    "a": 2
})

def foc(p):
    """Compute first order condition"""
    d = demandfn.demand(p)
    zero = 1 - (p - demandfn.cost) * (1 - d) / demandfn.mu
    return np.squeeze(zero)

def foc_monopoly(p):
    """Compute first order condition of a monopolist"""
    d = demandfn.demand(p)
    d1 = np.flip(d)
    p1 = np.flip(p)
    zero = 1 - (p - demandfn.cost) * (1 - d) / demandfn.mu + (p1 - demandfn.cost) * d1 / demandfn.mu
    return np.squeeze(zero)
def compute_p_competitive_monopoly():
    """Computes competitive and monopoly prices"""
    p0 = np.ones((1, 2)) * 1 * demandfn.cost
    p_competitive = fsolve(foc, p0)
    p_monopoly = fsolve(foc_monopoly, p0)
    return p_competitive, p_monopoly
compute_p_competitive_monopoly()

(array([1.47292666, 1.47292666]), array([1.92498092, 1.92498092]))

In [2]:
from gymnasium.spaces.box import Box
from gymnasium.spaces.discrete import Discrete

In [3]:
from environments.SimpleOligopolyEnv import SimpleOligopolyEnv

env_conf = {
    "max_price" : 2,
    "min_price" : 0,
    "num_sellers" : 2,
    "memory_size" : 5,
    "action_type" : "disc",
    "disc_action_size" : 5,
    "max_steps" : 10,
    "agent_ids" : ["agent0", "agent1"],
    "market" :{
    "demand_type" : "Logit",
        "logit" : {
        "a0" : 0,
        "cost" : 1,
        "mu" : 0.25,
        "a" : 2,
        },
        "linear" : None
    },
    "action_space": Discrete(5)
}
env = SimpleOligopolyEnv(seed = 0 , config=env_conf)
env.reset()

2023-10-21 03:05:26,070	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


({'agent0': array([1.56839255, 1.49715045, 1.0837855 , 2.22123056, 1.53417308,
         1.95417645, 1.41858114, 1.33589363, 1.65490475, 1.43980809]),
  'agent1': array([1.56839255, 1.49715045, 1.0837855 , 2.22123056, 1.53417308,
         1.95417645, 1.41858114, 1.33589363, 1.65490475, 1.43980809])},
 {'agent0': {}, 'agent1': {}})

In [3]:
env.step({"agent0":4, "agent1":4})

({'agent0': array([1.38237652, 1.93292072, 1.25659337, 2.16379079, 1.76177751,
         1.22317203, 1.28715066, 1.19606895, 2.37703518, 2.37703518]),
  'agent1': array([1.38237652, 1.93292072, 1.25659337, 2.16379079, 1.76177751,
         1.22317203, 1.28715066, 1.19606895, 2.37703518, 2.37703518])},
 {'agent0': 0.2112561, 'agent1': 0.2112561},
 {'agent0': False, 'agent1': False, '__all__': False},
 {'agent0': False, 'agent1': False, '__all__': False},
 {'agent0': {}, 'agent1': {}})

In [4]:
env.get_last_prices()

array([[2.37703518],
       [2.37703518]])

In [6]:
all_price = []
for _ in range(10):
    env.step({"agent0":randint(0,4), "agent1":randint(0,4)})
    all_price.append({"price":env.get_last_prices()})

In [7]:
all_price

[{'price': array([1.3599131, 1.3599131])},
 {'price': array([1.69895379, 1.69895379])},
 {'price': array([1.69895379, 1.3599131 ])},
 {'price': array([2.37703518, 1.3599131 ])},
 {'price': array([1.0208724 , 1.69895379])},
 {'price': array([1.3599131 , 2.37703518])},
 {'price': array([2.37703518, 1.3599131 ])},
 {'price': array([1.69895379, 1.3599131 ])},
 {'price': array([2.03799448, 1.3599131 ])},
 {'price': array([1.69895379, 2.37703518])}]

In [9]:
np.mean([step_prices['price'] for step_prices in all_price[-5:]],axis=0)

array([1.83457007, 1.76676193])